In [ ]:
# Multiarmed bandits 

# the casino machine has levers.
# each machine has a lever has an unknown probability distribution
# and can yield a reward potentially from it's probability distirbution. 
# ONE slot machine = one armed bandit
# Many slot machines = multi / k - armed bandits

# Goal: find out which slot machine gives us maximum cumulative reward over time.


# So at each time step, agent does an action (pulls slot machine arm + recieve reward) 
# the value of an action is: 
# Q_value(action) = sum of rewards from machine / total number of arms pulled on machine. 

# So the best slot machine is:
# Max(Q_value(action))

In [6]:
import gym_bandits
import gym
import numpy as np

In [3]:
env = gym.make("BanditTenArmedGaussian-v0")

[2018-10-25 11:56:21,292] Making new env: BanditTenArmedGaussian-v0


In [4]:
env.action_space # we have 10 arms to choose from, so action space is 10

Discrete(10)

# Exploration algorithms are used to solve the M-A-B problem

# 1. Epsilon Greedy exploration

In [7]:
num_rounds = 20000

# metrics to keep a track of 
count = np.zeros(10)
sum_rewards = np.zeros(10)
Q = np.zeros(10)

In [8]:
def epsilon_greedy(epsilon):
    rand = np.random.random()
    if rand < epsilon:
        action = env.action_space.sample()
    else:
        action = np.argmax(Q)
    
    return(action)

In [10]:
env.reset()

0

In [11]:
for i in range(num_rounds):
    # select arm using epsilon greedy
    arm = epsilon_greedy(0.5)
    # get reward
    observation, reward, done, info = env.step(arm)
    # update the count of that arm
    count[arm] += 1 
    # sum the rewards obtained from the arm
    sum_rewards[arm] += reward
    # calculate Q value which is the average rewards of the arm
    Q[arm] = sum_rewards[arm] / count[arm]

print("The optimal arm is {}".format(np.argmax(Q)))

The optimal arm is 1


# 2. Softmax exploration

In [12]:
# softmax exploration. / "Boltzman exploration"

# we select the best arm based on probability from botlzman distribution

# the probability of selecting an arm is given by equation:

# tau # is a parameter that is set
# t # temperature factor wich specify how many random arms can be explored. 
#   - when t is high all arms are explored equally
#   - when t is low, only arms with high rewards are chosen.

# Pt(arm) = exp(Qt(arm)/tau)  /  sum[exp(Qt(i)/tau)] 

In [17]:
import math as math
import random as random

In [19]:
def softmax(tau):
    total = sum([math.exp(val/tau) for val in Q]) # denominator
    probs = [math.exp(val/tau)/total for val in Q] # numerator  # val is Q value - i.e Qt(arm) 
    
    # probs = every Qt(arm) value has a probability associated with it. 
    
    threshold = random.random()
    cumulative_prob = 0.0 
    for i in range(len(probs)):           # for all probabilities for all actions...  
        cumulative_prob += probs[i]        # collect cumulative probability 
        if (cumulative_prob > threshold):  # once the cumulative probability reaches a threshold - return i.  
            return i
    return np.argmax(probs)   # else return the highest porbability

In [20]:
for i in range(num_rounds):
    # select arm using epsilon greedy
    arm = softmax(0.5)
    # get reward
    observation, reward, done, info = env.step(arm)
    # update the count of that arm
    count[arm] += 1 
    # sum the rewards obtained from the arm
    sum_rewards[arm] += reward
    # calculate Q value which is the average rewards of the arm
    Q[arm] = sum_rewards[arm] / count[arm]

print("The optimal arm is {}".format(np.argmax(Q)))

The optimal arm is 1


# Upper confidence bound algorithm

- sometimes actions which do not initially give good rewards may actually pay off in the long term. 
- this is based on the principle of optimism in the face of uncertainty. 
- we need to pull an arm multiple times to find out the true distribution of rewards the arm gives - despite priors
- the confidence interval species the range in which the mean reward of the value of the arm lies 
- between the LOWER confidence bound and the UPPER confidence bound is where the mean lies...


- UCB algorithm chooses arm / action that has highest sum of average reward and upper confidence bound. 
- whilst also updating arms reward and confidence bound after each action-reward

In [22]:
# Formula for upper condidence bound:
#  UCB = square root ( 2log(t)/ N(a))
# where N is number of times the action is performed / arm puled. 


# An action / arm is chosen with
# Arm = argmax [Q(a) + UCB]


In [23]:
def UCB(iters, num_possible_actions=10):
    '''
    There are 10 slot machines so num_possible_actions = 10 by default
    '''
    ucb = np.zeros(num_possible_actions)
    # explore all the arms
    if iters < num_possible_actions:
        return i
    else: 
        for arm in range(num_possible_actions):
            # calculate upper bound
            upper_bound = math.sqrt((2*math.log(sum(count))) / count[arm])
            # add upper bound to the Q value
            ucb[arm] = Q[arm] + upper_bound
        # retyrn the arm which has maximum value
        return(np.argmax(ucb))

In [25]:
num_rounds = 20000
count = np.zeros(10) # number of times arm is pulled - used within scope of UCB function. 
sum_rewards = np.zeros(10)
Q = np.zeros(10)

for i in range(num_rounds):
    arm = UCB(i, num_possible_actions=10)
    # get reward
    observation, reward, done, info = env.step(arm)
    # update count of arm
    count[arm] += 1
    # sum rewards obtained from the arm
    sum_rewards[arm] += reward
    # calculate q value which is the average reward of the arm
    Q[arm] = sum_rewards[arm] / count[arm]

print("The optimal arm is {}".format(np.argmax(Q)))

The optimal arm is 1


# Thompson sampling

In [26]:
num_rounds = 20000
count = np.zeros(10) # number of times arm is pulled - used within scope of UCB function. 
sum_rewards = np.zeros(10)
Q = np.zeros(10)

# initialise alpha and beta values:
alpha = np.ones(10)
beta = np.ones(10)

In [27]:
def thompson_sampling(alpha, beta, num_possible_actions=10):
    samples = [np.random.beta(alpha[i] + 1, beta[i] + 1) for i in range(num_possible_actions)]
    return np.argmax(samples)

In [28]:
for i in range(num_rounds):
    arm = thompson_sampling(alpha, beta, num_possible_actions=10)
    # get reward
    observation, reward, done, info = env.step(arm)
    # update count of arm
    count[arm] += 1
    # sum rewards obtained from the arm
    sum_rewards[arm] += reward
    # calculate q value which is the average reward of the arm
    Q[arm] = sum_rewards[arm] / count[arm]
    
    if reward > 0 : 
        alpha[arm] += 1
    else:
        beta[arm] += 1

print("The optimal arm is {}".format(np.argmax(Q)))

The optimal arm is 1
